# Limpeza da Base de Dados para extrair país, cidade, tipo de afastamento, patente e período de afastamento.

In [7]:
import re
import pandas as pd
from bs4 import BeautifulSoup

In [9]:
# Carregando os dados
dados_diario_uniao_militares_afastamentos = pd.read_excel(r"C:\Users\paula\OneDrive\IPEA\Militares_2\bases\dados_diario_uniao_militares_afastamentos.xlsx")

# Armazenando uma cópia do texto original em uma coluna temporária
dados_diario_uniao_militares_afastamentos['Texto_Reduzido'] = dados_diario_uniao_militares_afastamentos['Texto']

# Convertendo o texto para minúsculas apenas para o filtro
dados_diario_uniao_militares_afastamentos['Texto_filtro'] = dados_diario_uniao_militares_afastamentos['Texto'].str.lower()

# Filtrando o DataFrame para manter apenas as observações que contenham as palavras específicas
filtro = dados_diario_uniao_militares_afastamentos['Texto_filtro'].str.contains('nomear|autorizar|designar', case=False)
dados_filtrados_total = dados_diario_uniao_militares_afastamentos[filtro].copy()

# Removendo a coluna de filtro
dados_filtrados_total.drop(columns=['Texto_filtro'], inplace=True)

# Removendo linhas onde a coluna artCategory contém "UNIVERSIDADE"
dados_filtrados_total = dados_filtrados_total[~dados_filtrados_total['artCategory'].str.contains('UNIVERSIDADE', case=False, na=False)]

In [10]:
# Função para extrair o texto das tags <p> com as classes 'identifica pdf-CENTER' e 'identifica'
def extrair_identificacao(texto_html):
    soup = BeautifulSoup(texto_html, 'html.parser')
    identificacoes = soup.find_all('p', class_=lambda x: x and 'identifica' in x.split())
    texto_identificacao = ''
    for identificacao in identificacoes:
        texto_identificacao += identificacao.get_text(strip=True) + '\n'
        identificacao.decompose()  # Remover a identificação do texto original
    return texto_identificacao.strip()

# Função para extrair o texto reduzido, sem a identificação
def extrair_texto_reduzido(texto_html):
    soup = BeautifulSoup(texto_html, 'html.parser')
    # Remover as tags <p> com as classes 'identifica pdf-CENTER' e 'identifica' do texto
    for identificacao in soup.find_all('p', class_=lambda x: x and 'identifica' in x.split()):
        identificacao.decompose()
    texto_reduzido = soup.get_text(separator='\n', strip=True)
    return texto_reduzido.strip()

# Aplicar as funções para criar as variáveis Portaria_Especifica e Texto_Reduzido
dados_filtrados_total['Portaria_Especifica'] = dados_filtrados_total['Texto'].apply(extrair_identificacao)
dados_filtrados_total['Texto_Reduzido'] = dados_filtrados_total['Texto'].apply(extrair_texto_reduzido)

In [11]:
# Salvando os resultados em um arquivo Excel
pasta_base = "C:/Users/paula/OneDrive/IPEA/Militares_2/bases"
dados_filtrados_total.to_excel(f"{pasta_base}/dados_filtrados_total.xlsx", index=False)

### Filtrando a coluna para que as portarias repitidas em uma unica célula sejam consideradas parte do df

In [12]:
# Função para contar o número de ocorrências de 'nomear', 'autorizar' ou 'designar' no texto original
def contar_ocorrencias_reduzido(texto):
    padrao = re.compile(r'\b(nomear|autorizar|designar|autoriza)\b', re.IGNORECASE)
    return len(re.findall(padrao, texto))

# Adicionando uma nova coluna com o número de ocorrências
dados_filtrados_total['n_contagem_texto_original'] = dados_filtrados_total['Texto_Reduzido'].apply(contar_ocorrencias_reduzido)

# Salvando os resultados em um arquivo Excel
pasta_base = "C:/Users/paula/OneDrive/IPEA/Militares_2/bases"
dados_filtrados_total.to_excel(f"{pasta_base}/dados_filtrados_total.xlsx", index=False)

In [13]:
# Padrao da Divisao de Portarias 
pattern = r'(O\s+(?:CHEFE\s+DO\s+ESTADO-MAIOR\s+DA\s+ARMADA|MINISTRO\s+DE\s+ESTADO\s+DA\s+DEFESA|CHEFE\s+DO\s+ESTADO-MAIOR\s+CONJUNTO\s+DAS\s+FORÇAS\s+ARMADAS|DIRETOR-GERAL\s+DO\s+MATERIAL\s+DA\s+MARINHA|SECRETÁRIO-GERAL\s+DO\s+MINISTÉRIO\s+DA\s+DEFESA|DIRETOR-GERAL\s+DE\s+DESENVOLVIMENTO\s+NUCLEAR\s+E\s+TECNOLÓGICO\s+DA\s+MARINHA|COMANDANTE\s+DA\s+MARINHA|DIRETOR-GERAL\s+DE\s+NAVEGAÇÃO|COMANDANTE\s+DO\s+EXÉRCITO|COMANDANTE\s+DA\s+ESCOLA\s+SUPERIOR\s+DE\s+GUERRA|CHEFE\s+DO\s+ESTADO-MAIOR\s+DA\s+AERONÁUTICA|CHEFE\s+DE\s+GABINETE\s+DO\s+MINISTRO\s+DE\s+ESTADO\s+DA\s+DEFESA|COMANDANTE\s+DA\s+AERONÁUTICA))'

# Função para separar as portarias
def split_portarias(df, text_column, pattern):
    rows = []
    for index, row in df.iterrows():
        text = row[text_column]
        # Encontrar todas as correspondências do padrão
        matches = list(re.finditer(pattern, text))
        
        if not matches:
            # Se não houver correspondências, adicionar a linha original
            rows.append(row)
            continue
        
        # Adicionar a última posição para capturar o texto final
        positions = [match.start() for match in matches] + [len(text)]
        
        for i in range(len(positions) - 1):
            start = positions[i]
            end = positions[i + 1]
            portaria = text[start:end].strip()
            new_row = row.copy()
            new_row[text_column] = portaria
            rows.append(new_row)
    return pd.DataFrame(rows)

In [15]:
# Atualizar o DataFrame com as novas linhas
dados_filtrados_split = split_portarias(dados_filtrados_total, 'Texto_Reduzido', pattern)

In [17]:
# Função para contar o número de ocorrências de 'nomear', 'autorizar' ou 'designar' nos novos textos
def contar_ocorrencias_reduzido(texto):
    padrao = re.compile(r'\b(nomear|autorizar|designar|autoriza)\b', re.IGNORECASE)
    return len(re.findall(padrao, texto))

# Adicionando uma nova coluna com o número de ocorrências
dados_filtrados_split['n_contagem_texto_split'] = dados_filtrados_split['Texto_Reduzido'].apply(contar_ocorrencias_reduzido)

In [18]:
# Salvando os resultados em um arquivo Excel
pasta_base = "C:/Users/paula/OneDrive/IPEA/Militares_2/bases"
dados_filtrados_split.to_excel(f"{pasta_base}/dados_filtrados_split.xlsx", index=False)

### Limpando a variavel Texto_Reduzido

In [22]:
# Filtrar linhas com contagem diferente de zero
dados_filtrados_split = dados_filtrados_split[dados_filtrados_split['n_contagem_texto_split'] != 0]

# Remover quebras de linha do Texto_Reduzido e substituir por espaços
dados_filtrados_split['Texto_Reduzido'] = dados_filtrados_split['Texto_Reduzido'].str.replace('\n', ' ')

In [23]:
# Salvando os resultados em um arquivo Excel
pasta_base = "C:/Users/paula/OneDrive/IPEA/Militares_2/bases"
dados_filtrados_split.to_excel(f"{pasta_base}/base_afastamentos_militares_final.xlsx", index=False)

### Tratar Base Final e Transformar em texto para Treinamento do Modelo

In [24]:
# Carrega a base de dados
caminho_arquivo = "C:/Users/paula/OneDrive/IPEA/Militares_2/bases/base_afastamentos_militares_final.xlsx"
base_afastamentos_militares_final = pd.read_excel(caminho_arquivo)

# Concatenar todos os textos tratados em uma única string
texto_unico = '\n'.join(filter(lambda texto: texto is not None, base_afastamentos_militares_final['Texto_Reduzido']))

# Escolha um nome para o arquivo de texto único
nome_arquivo_txt = "C:/Users/paula/OneDrive/IPEA/Militares_2/texto_tratado_unico_final.txt"

# Salvar a string em um arquivo de texto
with open(nome_arquivo_txt, 'w', encoding='utf-8') as arquivo_txt:
    arquivo_txt.write(texto_unico)

print("Base de dados tratada e arquivo de texto único gerados com sucesso.")

Base de dados tratada e arquivo de texto único gerados com sucesso.


### Criar as bases separadas: NOMEAR, DESIGNAR E AUTORIZAR

In [26]:
# Carregar a base de dados tratada
caminho_arquivo_tratado = "C:/Users/paula/OneDrive/IPEA/Militares_2/bases/base_afastamentos_militares_final.xlsx"
base_tratada = pd.read_excel(caminho_arquivo_tratado)

# Função para verificar se uma célula contém duas ou mais palavras-chave diferentes
def contem_multiplas_ocorrencias_diferentes(texto):
    palavras_chave = ['AUTORIZAR', 'AUTORIZA', 'NOMEAR', 'DESIGNAR']
    ocorrencias = set()
    
    for palavra in palavras_chave:
        if re.search(rf'\b{palavra}\b', texto, re.IGNORECASE):
            ocorrencias.add(palavra)
    
    return len(ocorrencias) > 1

# Aplicar a função para encontrar células com múltiplas ocorrências diferentes
multiplas_ocorrencias_diferentes = base_tratada[base_tratada['Texto_Reduzido'].apply(contem_multiplas_ocorrencias_diferentes)]

# Exibir o resultado
print(f"Número de células com duas ou mais ocorrências diferentes: {len(multiplas_ocorrencias_diferentes)}")
print(multiplas_ocorrencias_diferentes[['Texto']])

Número de células com duas ou mais ocorrências diferentes: 52
                                                   Texto
159    <p class='identifica pdf-CENTER'>PORTARIAS DE ...
204    <p class='identifica pdf-CENTER'>PORTARIAS GAB...
267    <p class='identifica pdf-CENTER'>PORTARIAS SPE...
957    <p class='identifica pdf-CENTER'>PORTARIAS DAC...
1459   <p class='identifica pdf-CENTER'>PORTARIAS DE ...
1656   <p class='identifica pdf-CENTER'>DESPACHOS DO ...
2062   <p class='identifica pdf-CENTER'>PORTARIAS DE ...
2662   <p class='identifica pdf-CENTER'>Nº924 - NOMEA...
2967   <p class='identifica pdf-CENTER'>PORTARIAS DE ...
3121   <p class='identifica pdf-CENTER'>ANEXO A</p>\n...
3122   <p class='identifica pdf-CENTER'>ANEXO B</p>\n...
3124   <p class='identifica pdf-CENTER'>ANEXO D</p>\n...
3125   <p class='identifica pdf-CENTER'>ANEXO E</p>\n...
3126   <p class='identifica pdf-CENTER'>ANEXO F</p>\n...
3471   <p class='identifica pdf-CENTER'>PORTARIAS DE ...
3477   <p class='identific

In [ ]:

# Função para filtrar o texto com base em palavras-chave
def filtrar_por_palavra_chave(df, palavra_chave):
    return df[df['Texto'].str.contains(palavra_chave, case=False, na=False)]

# Filtra as portarias que contêm "NOMEAR"
base_nomear = filtrar_por_palavra_chave(base_tratada, 'NOMEAR')

# Filtra as portarias que contêm "DESIGNAR"
base_designar = filtrar_por_palavra_chave(base_tratada, 'DESIGNAR')

# Filtra as portarias que contêm "AUTORIZAR" ou "AUTORIZA"
base_autorizar = base_tratada[base_tratada['Texto'].str.contains('AUTORIZAR|AUTORIZA', case=False, na=False)]

# Remove duplicatas após a filtragem para evitar contar linhas repetidas
base_filtrada_unica = pd.concat([base_nomear, base_designar, base_autorizar]).drop_duplicates()

# Salvar as novas bases de dados
caminho_base_nomear = "C:/Users/paula/OneDrive/IPEA/Militares_2/base_nomear.xlsx"
caminho_base_designar = "C:/Users/paula/OneDrive/IPEA/Militares_2/base_designar.xlsx"
caminho_base_autorizar = "C:/Users/paula/OneDrive/IPEA/Militares_2/base_autorizar.xlsx"

base_nomear.to_excel(caminho_base_nomear, index=False)
base_designar.to_excel(caminho_base_designar, index=False)
base_autorizar.to_excel(caminho_base_autorizar, index=False)

# Calcular as perdas de linhas
total_linhas_original = len(base_tratada)
total_linhas_filtradas_unicas = len(base_filtrada_unica)
linhas_perdidas = total_linhas_original - total_linhas_filtradas_unicas

# Imprimir resultados
print(f"Total de linhas na base original: {total_linhas_original}")
print(f"Total de linhas após filtragem (únicas): {total_linhas_filtradas_unicas}")
print(f"Linhas perdidas: {linhas_perdidas}")
print("Bases de dados filtradas foram salvas com sucesso.")